# Optimieren von Hyperparametern mit einem Sweepauftrag

Es gibt viele Machine Learning-Algorithmen, die Hyperparameter erfordern (Parameterwerte, die das Training beeinflussen, aber nicht anhand der Trainingsdaten selbst bestimmt werden können). Wenn Sie zum Beispiel ein Modell für logistische Regression trainieren, können Sie den Hyperparameter „Abgrenzungsrate“ verwenden, um einer Verzerrung des Modells entgegenzuwirken. Oder Sie können beim Trainieren eines Convolutional Neural Networks (CNN) Hyperparameter wie „Lernrate“ und „Batchgröße“ verwenden, um zu steuern, wie Gewichtungen während des Trainings angepasst werden und wie viele Datenelemente in den einzelnen Minibatches enthalten sind. Die Auswahl von Hyperparameterwerten kann sich erheblich auf die Leistung eines trainierten Modells oder auf die Zeit auswirken, die für das Trainieren des Modells aufgewendet werden muss. Häufig müssen Sie mehrere Kombinationen ausprobieren, um die optimale Lösung zu finden. 

## Vorbereitung

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Erstellen des Trainingsskripts
Die Hyperparameteroptimierung ist ideal, wenn Sie ein Machine Learning-Modell trainieren möchten, aber die Eingabeparameter variieren möchten. Sie müssen ein Trainingsskript erstellen, das einen Eingabeparameter erwartet, der einen der Hyperparameter des Algorithmus darstellt.

Führen Sie die folgenden Zellen aus, um den Ordner **src** und das Trainingsskript zu erstellen.

Beachten Sie, dass das Trainingsskript zwei Eingabeparameter erwartet:

- `--training_data`, der eine Zeichenfolge erwartet. Sie geben den Pfad zu einer registrierten Datenressource als die Eingabetrainingsdaten an.
- `--reg_rate`, der eine Zahl erwartet, aber den Standardwert `0.01` hat. Sie verwenden diesen Eingabeparameter für die Hyperparameteroptimierung.

In [ ]:
## Konfigurieren und Ausführen eines Befehlsauftrags

Führen Sie die folgende Zelle aus, um ein Klassifizierungsmodell zur Vorhersage von Diabetes zu trainieren. Das Modell wird trainiert, indem das Skript **train\.py** ausgeführt wird, das sich im Ordner **src** befindet. Es verwendet die registrierte Datenressource `diabetes-data` als Trainingsdaten. 

- `code`: gibt den Ordner an, der das auszuführende Skript enthält.
- `command`: gibt an, was genau ausgeführt werden soll.
- `environment`: gibt die erforderlichen Pakete an, die in der Compute-Instanz installiert werden müssen, ehe der Befehl ausgeführt wird.
- `compute`: gibt die Compute-Instanz an, die zum Ausführen des Befehls verwendet werden soll.
- `display_name`: der Name des einzelnen Auftrags.
- `experiment_name`: der Name des Experiments, zu dem der Auftrag gehört.

Beachten Sie, dass der Befehlsauftrag das Trainingsskript nur einmal mit der Abgrenzungsrate `0.1` ausführt. Bevor Sie einen Sweepauftrag ausführen, um Hyperparameter zu optimieren, empfiehlt ein Test, ob Ihr Skript mit einem Befehlsauftrag wie erwartet funktioniert.

In [ ]:
## Definieren des Suchbereichs

Wenn Ihr Befehlsauftrag erfolgreich abgeschlossen wurde, können Sie einen Sweepauftrag konfigurieren und ausführen. 

Zunächst müssen Sie den Suchbereich für Ihren Hyperparameter angeben. Um drei Modelle mit jeweils einer anderen Abgrenzungsrate (`0.01`, `0.1` oder `1`) zu trainieren, können Sie den Suchbereich mit dem Hyperparameter `Choice` definieren. 

## Konfigurieren und Übermitteln des Sweepauftrags

Sie verwenden die Sweepfunktion, um die Hyperparameteroptimierung für Ihr Trainingsskript durchzuführen. Zum Konfigurieren eines Sweepauftrags müssen Sie Folgendes angeben:

- `compute`: Name des Computeziels, in dem der Auftrag ausgeführt werden soll.
- `sampling_algorithm`:der Samplingalgorithmus für Hyperparameter, der im Suchbereich verwendet werden soll. Zulässige Werte sind `random`, `grid` und `bayesian`.
- `primary_metric`: der Name der primären Metrik, die von jedem Testauftrag gemeldet wird. Die Metrik muss im Trainingsskript des Benutzers mithilfe von `mlflow.log_metric()` mit dem gleichen entsprechenden Metriknamen protokolliert werden.
- `goal`: das Optimierungsziel von `primary_metric`. Die zulässigen Werte sind `maximize` und `minimize`.
- `limits`: Grenzwerte für den Sweepauftrag. Beispiel: die maximale Anzahl von Testversionen oder Modellen, die Sie trainieren möchten.

Beachten Sie, dass der Befehlsauftrag als Basis für den Sweepauftrag dient. Die Konfiguration des Befehlsauftrags wird vom Sweepauftrag wiederverwendet.

In [ ]:
Führen Sie die folgende Zelle aus, um den Sweepauftrag zu übermitteln.

In [ ]:
Wenn der Auftrag abgeschlossen ist, navigieren Sie zur Auftragsübersicht. Auf der Registerkarte **Versuche** sehen Sie alle Modelle, die trainiert wurden, und wie sich die Bewertung von `Accuracy` für jeden von Ihnen ausprobierten Abgrenzungsratenwert unterscheidet.

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.